In [8]:
import nibabel as nib
from matplotlib import pyplot as plt
import torchio as tio

In [1]:
from tqdm import trange
from time import sleep

for i in trange(100):
    sleep(1)

100%|██████████| 100/100 [01:40<00:00,  1.00s/it]


In [8]:
import torch
t = torch.tensor([[9], [9]])
y = torch.tensor([1,2])


torch.gather(t, dim=1, index= y)

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [8]:
import pandas as pd
n=6
df = pd.read_csv("data/survival/survival_data.csv")
df['survival_bin'] = pd.qcut(df['survival_months'], n, labels = [x for x in range(n)])
df['survival_bin'].value_counts()

df.to_csv("data/survival/TCGA_survival_bins.csv")